In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import itertools
import os

### Read and Combine the txt files

In [4]:
# Path to the folder containing the text files
folder_path = r"/content/drive/MyDrive/CS_Finance_Data"

# List all files in the folder
file_list = [f for f in os.listdir(folder_path) if f.endswith('.txt')] ## List Comprehension

# Initialize an empty list to store DataFrames
dataframes = []

# Read each file and append the DataFrame to the list
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path, sep='^', header=0, engine='python', encoding='latin1', on_bad_lines='skip')
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the first few rows of the combined DataFrame
combined_df.head()

,RSSD9001,RSSD9999,RSSD9007,RSSD9008,RSSD9132,RSSD9032,RSSD9146,BHBC3368,BHBC3402,BHBC3516,...,BHCANC99,BHCKFT42,BHCKFT43,BHCKFT44,BHSPFT42,BHSPFT43,BHSPFT44,BHSPMZ36,BHSPNK60,BHCKMG95
0,1020180,20041231,20040823,20051118,551111,9,10.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1020201,20041231,20001028,20051230,551111,2,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1020340,20041231,20041230,20050527,551111,7,30.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1020395,20041231,20040211,20081230,551111,6,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1020582,20041231,19980101,20060228,551111,7,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Subset some columns:

In [18]:
smalldf = combined_df[['RSSD9001','RSSD9017', 'RSSD9032', 'RSSD9146', 'RSSD9999', 'BHCK4107', 'BHCK4074', 'BHCK2170']]
smalldf.head()

,RSSD9001,RSSD9017,RSSD9032,RSSD9146,RSSD9999,BHCK4107,BHCK4074,BHCK2170
0,1020180,BREMER FINANCIAL CORPORATION,9,10.0,20041231,291916.0,203077.0,6141519.0
1,1020201,HSBC USA INC.,2,1.0,20041231,4078746.0,2741737.0,141049972.0
2,1020340,"HARRIS BANKCORP, INC.",7,30.0,20041231,1179568.0,832167.0,34389852.0
3,1020395,SOUTHERN NATIONAL CORPORATION,6,1.0,20041231,12248.0,7630.0,226549.0
4,1020582,"WCN BANCORP, INC.",7,1.0,20041231,11035.0,8331.0,250744.0


In [19]:
name_dict = {'RSSD9001':'RSSD ID',
             'RSSD9017':'Firm Legal Name',
             'RSSD9032':'FR District Code',
             'RSSD9146':'Bank Count',
             'RSSD9999':'Reporting Date',
             'BHCK4107':'Interest Income',
             'BHCK4074':'Net Interest Income',
             'BHCK2170':'Total Assets',
             }
smalldf = smalldf.rename(columns = name_dict)
print(smalldf.shape)
smalldf.head()

(243757, 8)


,RSSD ID,Firm Legal Name,FR District Code,Bank Count,Reporting Date,Interest Income,Net Interest Income,Total Assets
0,1020180,BREMER FINANCIAL CORPORATION,9,10.0,20041231,291916.0,203077.0,6141519.0
1,1020201,HSBC USA INC.,2,1.0,20041231,4078746.0,2741737.0,141049972.0
2,1020340,"HARRIS BANKCORP, INC.",7,30.0,20041231,1179568.0,832167.0,34389852.0
3,1020395,SOUTHERN NATIONAL CORPORATION,6,1.0,20041231,12248.0,7630.0,226549.0
4,1020582,"WCN BANCORP, INC.",7,1.0,20041231,11035.0,8331.0,250744.0


In [20]:
smalldf['Reporting Date'] = pd.to_datetime(smalldf['Reporting Date'], format = "%Y%m%d")

### Define Quarters and Years


In [21]:
# Define Quarter

smalldf['Quarter'] = smalldf['Reporting Date'].dt.quarter
# smalldf.head()

In [22]:
# Define Year

smalldf['Year'] = smalldf['Reporting Date'].dt.year
smalldf.head()

,RSSD ID,Firm Legal Name,FR District Code,Bank Count,Reporting Date,Interest Income,Net Interest Income,Total Assets,Quarter,Year
0,1020180,BREMER FINANCIAL CORPORATION,9,10.0,2004-12-31,291916.0,203077.0,6141519.0,4,2004
1,1020201,HSBC USA INC.,2,1.0,2004-12-31,4078746.0,2741737.0,141049972.0,4,2004
2,1020340,"HARRIS BANKCORP, INC.",7,30.0,2004-12-31,1179568.0,832167.0,34389852.0,4,2004
3,1020395,SOUTHERN NATIONAL CORPORATION,6,1.0,2004-12-31,12248.0,7630.0,226549.0,4,2004
4,1020582,"WCN BANCORP, INC.",7,1.0,2004-12-31,11035.0,8331.0,250744.0,4,2004


## Quaterization

In [23]:
def quarterize(df, cols_to_process, dt_column):
  """
  Function to quarterizes the columns of a dataframe.
    df: has to be sorted by year and quarters
    cols_to_process: list of columns to quarterize
    dt_column: string
              column name of the date column

  """
  # indices is considered to be default
  fq_ids = df[(df[dt_column].dt.month == 3) & (df[dt_column].dt.day == 31)].index
  for col in cols_to_process:
    df[f"Q{col}"] = df[col].diff()
    df.loc[fq_ids, f"Q{col}"] = df.loc[fq_ids, col]
  return df

In [24]:
ls_dfs = []
for bank in smalldf['RSSD ID'].unique():
  OneBank = smalldf[smalldf["RSSD ID"] == bank].sort_values(by=['Reporting Date'])
  tmp_df = quarterize(OneBank, ["Interest Income", "Net Interest Income"], dt_column="Reporting Date")
  ls_dfs.append(tmp_df)

Quarterized_dfs = pd.concat(ls_dfs, ignore_index=True)
Quarterized_dfs.head(10)

,RSSD ID,Firm Legal Name,FR District Code,Bank Count,Reporting Date,Interest Income,Net Interest Income,Total Assets,Quarter,Year,QInterest Income,QNet Interest Income
0,1020180,BREMER FINANCIAL CORPORATION,9,11.0,2004-03-31,68233.0,46844.0,5721282.0,1,2004,68233.0,46844.0
1,1020180,BREMER FINANCIAL CORPORATION,9,11.0,2004-06-30,138086.0,95452.0,5870480.0,2,2004,69853.0,48608.0
2,1020180,BREMER FINANCIAL CORPORATION,9,10.0,2004-09-30,212489.0,147314.0,5963700.0,3,2004,74403.0,51862.0
3,1020180,BREMER FINANCIAL CORPORATION,9,10.0,2004-12-31,291916.0,203077.0,6141519.0,4,2004,79427.0,55763.0
4,1020180,BREMER FINANCIAL CORPORATION,9,10.0,2005-03-31,81656.0,54637.0,6230236.0,1,2005,81656.0,54637.0
5,1020180,BREMER FINANCIAL CORPORATION,9,10.0,2005-06-30,170175.0,110844.0,6450716.0,2,2005,88519.0,56207.0
6,1020180,BREMER FINANCIAL CORPORATION,9,10.0,2005-09-30,265290.0,169488.0,6336699.0,3,2005,95115.0,58644.0
7,1020180,BREMER FINANCIAL CORPORATION,9,9.0,2005-12-31,362347.0,227516.0,6555895.0,4,2005,97057.0,58028.0
8,1020180,BREMER FINANCIAL CORPORATION,9,9.0,2006-03-31,100170.0,56954.0,6480680.0,1,2006,100170.0,56954.0
9,1020180,BREMER FINANCIAL CORPORATION,9,9.0,2006-06-30,206822.0,115120.0,6638952.0,2,2006,106652.0,58166.0


### Normalization

In [25]:
# Normalize Components
Quarterized_dfs['Normalized Interest Income per Quarter'] = Quarterized_dfs['QInterest Income']/Quarterized_dfs['Total Assets']

Quarterized_dfs['Normalized Net Interest Income per Quarter'] = Quarterized_dfs['QNet Interest Income']/Quarterized_dfs['Total Assets']

Quarterized_dfs.head()

,RSSD ID,Firm Legal Name,FR District Code,Bank Count,Reporting Date,Interest Income,Net Interest Income,Total Assets,Quarter,Year,QInterest Income,QNet Interest Income,Normalized Interest Income per Quarter,Normalized Net Interest Income per Quarter
0,1020180,BREMER FINANCIAL CORPORATION,9,11.0,2004-03-31,68233.0,46844.0,5721282.0,1,2004,68233.0,46844.0,0.011926,0.008188
1,1020180,BREMER FINANCIAL CORPORATION,9,11.0,2004-06-30,138086.0,95452.0,5870480.0,2,2004,69853.0,48608.0,0.011899,0.008280
2,1020180,BREMER FINANCIAL CORPORATION,9,10.0,2004-09-30,212489.0,147314.0,5963700.0,3,2004,74403.0,51862.0,0.012476,0.008696
3,1020180,BREMER FINANCIAL CORPORATION,9,10.0,2004-12-31,291916.0,203077.0,6141519.0,4,2004,79427.0,55763.0,0.012933,0.009080
4,1020180,BREMER FINANCIAL CORPORATION,9,10.0,2005-03-31,81656.0,54637.0,6230236.0,1,2005,81656.0,54637.0,0.013106,0.008770


### We bring our Macroeconomic Variables

In [28]:
gdp_df = pd.read_csv(r"/content/drive/MyDrive/CS_Finance_Data/MacroEconomic_Variables/GDP.csv", parse_dates=True)
gdp_df['DATE'] = pd.to_datetime(gdp_df['DATE'])
print(gdp_df.shape)
gdp_df.head()

(81, 2)


,DATE,GDP
0,2004-01-01,11923.447
1,2004-04-01,12112.815
2,2004-07-01,12305.307
3,2004-10-01,12527.214
4,2005-01-01,12767.286


In [29]:
uner_df = pd.read_csv(r"/content/drive/MyDrive/CS_Finance_Data/MacroEconomic_Variables/UNRATE.csv")
uner_df['DATE'] = pd.to_datetime(uner_df['DATE'])
print(uner_df.shape)
uner_df.head()

(82, 2)


,DATE,UNRATE
0,2004-01-01,5.700000
1,2004-04-01,5.600000
2,2004-07-01,5.433333
3,2004-10-01,5.433333
4,2005-01-01,5.300000


In [30]:
df_gdp_uner = pd.merge(gdp_df,uner_df,on='DATE')
print(df_gdp_uner.shape)
df_gdp_uner.head()

(81, 3)


,DATE,GDP,UNRATE
0,2004-01-01,11923.447,5.700000
1,2004-04-01,12112.815,5.600000
2,2004-07-01,12305.307,5.433333
3,2004-10-01,12527.214,5.433333
4,2005-01-01,12767.286,5.300000


In [31]:
df_gdp_uner.rename(columns={'DATE': 'date'}, inplace=True)

In [33]:
def update_quart_enddate_shift(df):
    # Define a dictionary to map the month to the new date
    quarter_end_dates = {1: (12, 31), 4: (3, 31), 7: (6, 30), 10: (9, 30)}

    # Apply the mapping to update the dates
    for month, (new_month, new_day) in quarter_end_dates.items():
        df.loc[df['date'].dt.month == month, 'date'] = df['date'].apply(
            lambda x: x.replace(month=new_month, day=new_day) if x.month == month else x
        )

    # offset the years by 1 for those whose month is december
    tmp_ids = df[df["date"].dt.month == 12].index
    df.loc[tmp_ids, "date"] = df.loc[tmp_ids, "date"] - pd.DateOffset(years=1)

    # drop the first row df_gdp_uner
    df.drop(df.index[0], inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [34]:
df_gdp_uner =  update_quart_enddate_shift(df_gdp_uner)
df_gdp_uner.head()

,date,GDP,UNRATE
0,2004-03-31,12112.815,5.600000
1,2004-06-30,12305.307,5.433333
2,2004-09-30,12527.214,5.433333
3,2004-12-31,12767.286,5.300000
4,2005-03-31,12922.656,5.100000


### US Department of Treasury Dataset

In [41]:
rates_df = pd.read_csv(r"/content/drive/MyDrive/CS_Finance_Data/MacroEconomic_Variables/yield-curve-rates-2004-2024.csv", encoding='latin1')
rates_df.head()

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,07/15/2024,5.48,5.51,5.43,5.40,5.23,4.85,4.44,4.23,4.13,4.16,4.23,4.56,4.46
1,07/12/2024,5.47,5.52,5.43,5.41,5.23,4.87,4.45,4.22,4.10,4.13,4.18,4.50,4.39
2,07/11/2024,5.48,5.53,5.44,5.41,5.25,4.91,4.50,4.26,4.13,4.15,4.20,4.51,4.41
3,07/10/2024,5.46,5.50,5.46,5.46,5.33,5.01,4.62,4.38,4.24,4.24,4.28,4.58,4.47
4,07/09/2024,5.45,5.51,5.46,5.46,5.34,5.02,4.62,4.37,4.24,4.25,4.30,4.59,4.49


In [42]:
# Changing Date to Date-time Format
rates_df['Date'] = pd.to_datetime(rates_df['Date'].astype(str), format='mixed', utc=True)
rates_df.head()

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,2024-07-15 00:00:00+00:00,5.48,5.51,5.43,5.40,5.23,4.85,4.44,4.23,4.13,4.16,4.23,4.56,4.46
1,2024-07-12 00:00:00+00:00,5.47,5.52,5.43,5.41,5.23,4.87,4.45,4.22,4.10,4.13,4.18,4.50,4.39
2,2024-07-11 00:00:00+00:00,5.48,5.53,5.44,5.41,5.25,4.91,4.50,4.26,4.13,4.15,4.20,4.51,4.41
3,2024-07-10 00:00:00+00:00,5.46,5.50,5.46,5.46,5.33,5.01,4.62,4.38,4.24,4.24,4.28,4.58,4.47
4,2024-07-09 00:00:00+00:00,5.45,5.51,5.46,5.46,5.34,5.02,4.62,4.37,4.24,4.25,4.30,4.59,4.49


In [43]:
# Giving the Year and Quarter each entry
rates_df['Year'] = rates_df['Date'].dt.year
rates_df['Quarter'] = rates_df['Date'].dt.quarter
rates_df.sort_values('Date').head()

,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr,Year,Quarter
5138,2004-01-02 00:00:00+00:00,0.88,NaN,0.93,NaN,1.02,1.31,1.94,2.47,3.36,3.90,4.38,5.21,NaN,2004,1
5137,2004-01-05 00:00:00+00:00,0.88,NaN,0.91,NaN,1.05,1.35,1.95,2.51,3.39,3.92,4.41,5.23,NaN,2004,1
5136,2004-01-06 00:00:00+00:00,0.88,NaN,0.91,NaN,1.03,1.30,1.84,2.38,3.26,3.80,4.29,5.13,NaN,2004,1
5135,2004-01-07 00:00:00+00:00,0.88,NaN,0.91,NaN,1.02,1.29,1.84,2.36,3.25,3.76,4.27,5.11,NaN,2004,1
5134,2004-01-08 00:00:00+00:00,0.87,NaN,0.88,NaN,1.01,1.29,1.85,2.37,3.24,3.76,4.27,5.12,NaN,2004,1


In [44]:
finalrates = rates_df.groupby(['Year', 'Quarter']).mean()
finalrates = finalrates.drop(columns=["Date"])
finalrates.head(20)

1 Mo  2 Mo      3 Mo  4 Mo      6 Mo      1 Yr      2 Yr  \
Year Quarter                                                                 
2004 1        0.909677   NaN  0.933710   NaN  1.003548  1.221452  1.686613   
     2        0.954516   NaN  1.096129   NaN  1.359677  1.775484  2.454677   
     3        1.370000   NaN  1.513906   NaN  1.788437  2.075000  2.556563   
     4        1.841774   NaN  2.047419   NaN  2.297581  2.472742  2.822419   
2005 1        2.360984   NaN  2.589180   NaN  2.876885  3.072459  3.454918   
     2        2.709687   NaN  2.926719   NaN  3.179688  3.337344  3.645938   
     3        3.228594   NaN  3.438281   NaN  3.707031  3.791094  3.957969   
     4        3.702131   NaN  3.913607   NaN  4.250492  4.288525  4.364590   
2006 1        4.358710   NaN  4.506613   NaN  4.657258  4.640323  4.604677   
     2        4.674921   NaN  4.830159   NaN  5.033651  5.021429  4.997937   
     3        4.953492   NaN  5.035079   NaN  5.172540  5.091111  4.929048   
     4        5.020161   NaN  5.031774   NaN  5.113387  4.988226  4.737581   
2007 1        5.107742   NaN  5.114032   NaN  5.133065  5.007581  4.760484   
     2        4.776719   NaN  4.872031   NaN  5.000000  4.934219  4.803750   
     3        4.278889   NaN  4.434286   NaN  4.607778  4.535079  4.390794   
     4        3.462097   NaN  3.491613   NaN  3.709194  3.635000  3.489677   
2008 1        2.130820   NaN  2.104426   NaN  2.158361  2.112951  2.029344   
     2        1.511094   NaN  1.650156   NaN  1.872500  2.065937  2.416406   
     3        1.392656   NaN  1.520312   NaN  1.863125  2.126406  2.360000   
     4        0.141290   NaN  0.327627   NaN  0.740968  0.989194  1.215484   

                  3 Yr      5 Yr      7 Yr     10 Yr     20 Yr     30 Yr  
Year Quarter                                                              
2004 1        2.163065  2.980968  3.501452  4.010000  4.877903       NaN  
     2        2.971935  3.720968  4.183871  4.597097  5.355645       NaN  
     3        2.919531  3.505625  3.922188  4.301406  5.068281       NaN  
     4        3.058871  3.493710  3.855000  4.175806  4.874839       NaN  
2005 1        3.623934  3.890820  4.098852  4.303607  4.764098       NaN  
     2        3.732344  3.872969  3.984688  4.158594  4.548750       NaN  
     3        3.989219  4.039375  4.108437  4.215156  4.507656       NaN  
     4        4.372295  4.391148  4.425738  4.488197  4.767541       NaN  
2006 1        4.582581  4.552419  4.555484  4.577097  4.774032  4.663056  
     2        4.987302  4.993175  5.018095  5.072698  5.290635  5.143016  
     3        4.868413  4.842222  4.850000  4.893492  5.085397  4.994286  
     4        4.648710  4.601774  4.603548  4.630645  4.832903  4.741935  
2007 1        4.679677  4.643387  4.649677  4.679032  4.890806  4.796452  
     2        4.763594  4.760781  4.783750  4.846094  5.069063  4.988906  
     3        4.416508  4.512698  4.602857  4.738413  5.012540  4.948889  
     4        3.513387  3.797581  3.991613  4.266613  4.659032  4.612258  
2008 1        2.170984  2.751475  3.152131  3.665246  4.399016  4.412131  
     2        2.659063  3.155000  3.458281  3.881562  4.587031  4.574063  
     3        2.632969  3.112812  3.437031  3.862188  4.495625  4.449219  
     4        1.480968  2.172742  2.620806  3.234677  3.946290  3.661774

In [46]:
finalrates_df = finalrates.copy()
finalrates_df.reset_index(inplace=True)
finalrates_df.head()

,Year,Quarter,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,2004,1,0.909677,NaN,0.933710,NaN,1.003548,1.221452,1.686613,2.163065,2.980968,3.501452,4.010000,4.877903,NaN
1,2004,2,0.954516,NaN,1.096129,NaN,1.359677,1.775484,2.454677,2.971935,3.720968,4.183871,4.597097,5.355645,NaN
2,2004,3,1.370000,NaN,1.513906,NaN,1.788437,2.075000,2.556563,2.919531,3.505625,3.922188,4.301406,5.068281,NaN
3,2004,4,1.841774,NaN,2.047419,NaN,2.297581,2.472742,2.822419,3.058871,3.493710,3.855000,4.175806,4.874839,NaN
4,2005,1,2.360984,NaN,2.589180,NaN,2.876885,3.072459,3.454918,3.623934,3.890820,4.098852,4.303607,4.764098,NaN


In [56]:
cmev = pd.concat([finalrates_df, df_gdp_uner], axis = 1)
cmev.drop(columns=['Year', 'Quarter'], inplace=True)
cmev.head()

,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr,date,GDP,UNRATE
0,0.909677,NaN,0.933710,NaN,1.003548,1.221452,1.686613,2.163065,2.980968,3.501452,4.010000,4.877903,NaN,2004-03-31,12112.815,5.600000
1,0.954516,NaN,1.096129,NaN,1.359677,1.775484,2.454677,2.971935,3.720968,4.183871,4.597097,5.355645,NaN,2004-06-30,12305.307,5.433333
2,1.370000,NaN,1.513906,NaN,1.788437,2.075000,2.556563,2.919531,3.505625,3.922188,4.301406,5.068281,NaN,2004-09-30,12527.214,5.433333
3,1.841774,NaN,2.047419,NaN,2.297581,2.472742,2.822419,3.058871,3.493710,3.855000,4.175806,4.874839,NaN,2004-12-31,12767.286,5.300000
4,2.360984,NaN,2.589180,NaN,2.876885,3.072459,3.454918,3.623934,3.890820,4.098852,4.303607,4.764098,NaN,2005-03-31,12922.656,5.100000


In [57]:
Quarterized_dfs.head()

,RSSD ID,Firm Legal Name,FR District Code,Bank Count,Reporting Date,Interest Income,Net Interest Income,Total Assets,Quarter,Year,QInterest Income,QNet Interest Income,Normalized Interest Income per Quarter,Normalized Net Interest Income per Quarter
0,1020180,BREMER FINANCIAL CORPORATION,9,11.0,2004-03-31,68233.0,46844.0,5721282.0,1,2004,68233.0,46844.0,0.011926,0.008188
1,1020180,BREMER FINANCIAL CORPORATION,9,11.0,2004-06-30,138086.0,95452.0,5870480.0,2,2004,69853.0,48608.0,0.011899,0.008280
2,1020180,BREMER FINANCIAL CORPORATION,9,10.0,2004-09-30,212489.0,147314.0,5963700.0,3,2004,74403.0,51862.0,0.012476,0.008696
3,1020180,BREMER FINANCIAL CORPORATION,9,10.0,2004-12-31,291916.0,203077.0,6141519.0,4,2004,79427.0,55763.0,0.012933,0.009080
4,1020180,BREMER FINANCIAL CORPORATION,9,10.0,2005-03-31,81656.0,54637.0,6230236.0,1,2005,81656.0,54637.0,0.013106,0.008770


In [58]:
# selecting some columns of Quater
small_Quaterized_dfs = Quarterized_dfs[['RSSD ID', 'Firm Legal Name', 'Bank Count', 'Reporting Date', 'Year', 'Quarter',
                                        'Normalized Interest Income per Quarter', 'Normalized Net Interest Income per Quarter']]

small_Quaterized_dfs.head()

,RSSD ID,Firm Legal Name,Bank Count,Reporting Date,Year,Quarter,Normalized Interest Income per Quarter,Normalized Net Interest Income per Quarter
0,1020180,BREMER FINANCIAL CORPORATION,11.0,2004-03-31,2004,1,0.011926,0.008188
1,1020180,BREMER FINANCIAL CORPORATION,11.0,2004-06-30,2004,2,0.011899,0.008280
2,1020180,BREMER FINANCIAL CORPORATION,10.0,2004-09-30,2004,3,0.012476,0.008696
3,1020180,BREMER FINANCIAL CORPORATION,10.0,2004-12-31,2004,4,0.012933,0.009080
4,1020180,BREMER FINANCIAL CORPORATION,10.0,2005-03-31,2005,1,0.013106,0.008770


In [65]:
df = small_Quaterized_dfs.merge(cmev, left_on='Reporting Date', right_on='date', how='left')
df.drop(columns=['date'], inplace=True)
df.head()

,RSSD ID,Firm Legal Name,Bank Count,Reporting Date,Year,Quarter,Normalized Interest Income per Quarter,Normalized Net Interest Income per Quarter,1 Mo,2 Mo,...,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr,GDP,UNRATE
0,1020180,BREMER FINANCIAL CORPORATION,11.0,2004-03-31,2004,1,0.011926,0.008188,0.909677,NaN,...,1.221452,1.686613,2.163065,2.980968,3.501452,4.010000,4.877903,NaN,12112.815,5.600000
1,1020180,BREMER FINANCIAL CORPORATION,11.0,2004-06-30,2004,2,0.011899,0.008280,0.954516,NaN,...,1.775484,2.454677,2.971935,3.720968,4.183871,4.597097,5.355645,NaN,12305.307,5.433333
2,1020180,BREMER FINANCIAL CORPORATION,10.0,2004-09-30,2004,3,0.012476,0.008696,1.370000,NaN,...,2.075000,2.556563,2.919531,3.505625,3.922188,4.301406,5.068281,NaN,12527.214,5.433333
3,1020180,BREMER FINANCIAL CORPORATION,10.0,2004-12-31,2004,4,0.012933,0.009080,1.841774,NaN,...,2.472742,2.822419,3.058871,3.493710,3.855000,4.175806,4.874839,NaN,12767.286,5.300000
4,1020180,BREMER FINANCIAL CORPORATION,10.0,2005-03-31,2005,1,0.013106,0.008770,2.360984,NaN,...,3.072459,3.454918,3.623934,3.890820,4.098852,4.303607,4.764098,NaN,12922.656,5.100000


In [64]:
df.shape

(243757, 23)

In [66]:
df.isnull().sum()

RSSD ID                                            0
Firm Legal Name                                    0
Bank Count                                      9119
Reporting Date                                     0
Year                                               0
Quarter                                            0
Normalized Interest Income per Quarter        170337
Normalized Net Interest Income per Quarter    170340
1 Mo                                             473
2 Mo                                          194565
3 Mo                                             473
4 Mo                                          231244
6 Mo                                             473
1 Yr                                             473
2 Yr                                             473
3 Yr                                             473
5 Yr                                             473
7 Yr                                             473
10 Yr                                         